# Human Activity Recognition using TensorFlow (RNN LSTM)

## Code and Plots Location (GitHub Repository)

### Team Members: 

* Kannu Priya Arora (kannupriya@gmail.com)
      
* Sanatan Das (sanatanonline@gmail.com)


### GitHub: https://github.com/sanatanonline/compscix-433-assignments/tree/master/src/csx_433_7/project

## Introduction

**Project Overview** The data comprises of accelerometer and gyroscope signal data of day to day
activities performed by individuals. Project objective is to use Tensorflow to build a neural network
model for accurately classifying each activity into one of following 6 categories:
    
1) Walking

2) Walking upstairs

3) Walking downstairs

4) Sitting

5) Standing

6) Laying

**Success Metric:** Assuming target user of the model as company like Fitbit. The minimum accuracy that
company would expect is 90% to make it reliable technology for their end customer. We will target this
90% accuracy through recurrent neural networks (RNN). For long term reliability, the attempt is to build
deep long short term memory (LSTM) model.

## Overview of the Data

### Download the dataset

**https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip**

## Description of the data

The notes from the data source:

*The sensor signals (accelerometer and gyroscope) were pre-processed by applying noise filters and then sampled in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window). The sensor acceleration signal, which has gravitational and body motion components, was separated using a Butterworth low-pass filter into body acceleration and gravity. The gravitational force is assumed to have only low frequency components, therefore a filter with 0.3 Hz cutoff frequency was used.*

We are using the dataset as is from the input source considering the data is already cleaned up and ready to use.

## Code

### Import the libraries

In [ ]:
# Import the libraries numpy, matplotlib, tensorflow and metrics from sklearn
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import metrics

### Create some useful constants

In [ ]:
# These are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "WALKING",
    "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
]

# Preparing data set
DATASET_PATH = "HAR_data/"
TRAIN = "train/"
TEST = "test/"

### Load the data

In [ ]:
# Load "X" (the neural network's training and testing inputs)
def load_x(x_signals_paths):
    x_signals = []

    for signal_type_path in x_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        x_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()

    return np.transpose(np.array(x_signals), (1, 2, 0))


X_train_signals_paths = [
    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

x_train = load_x(X_train_signals_paths)
x_test = load_x(X_test_signals_paths)

if x_train.size != 0:
    print("x-train data is loaded successfully")

if x_test.size != 0:
    print("x-test data is loaded successfully")


# Load "y" (the neural network's training and testing outputs)
def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()

    # Subtract 1 to each output class for friendly 0-based indexing
    return y_ - 1


y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
y_test_path = DATASET_PATH + TEST + "y_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)

if y_train.size != 0:
    print("y-train data is loaded successfully")

if y_test.size != 0:
    print("y-test data is loaded successfully")

### Additional Parameters

In [ ]:
# Input Data

training_data_count = len(x_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(x_test)  # 2947 testing series
n_steps = len(x_train[0])  # 128 timesteps per series
n_input = len(x_train[0][0])  # 9 input parameters per timestep


# LSTM Neural Network's internal structure

n_hidden = 32  # Hidden layer num of features
n_classes = 6  # Total classes (should go up, or should go down)


# Training

# Initialize learning rate
learning_rate = 0.0025
# Loss amount
lambda_loss_amount = 0.0015
# Loop 300 times on the dataset
training_iters = training_data_count * 3
# Batch size
batch_size = 1500
# To show test set accuracy during training
display_iter = 30000


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(x_test.shape, y_test.shape, np.mean(x_test), np.std(x_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")


### Utility functions for training

In [ ]:
def lstm_rnn(_x, _weights, _biases):
    # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters.
    # Moreover, two LSTM cells are stacked which adds deepness to the neural network.

    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)
    _x = tf.transpose(_x, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _x = tf.reshape(_x, [-1, n_input])
    # new shape: (n_steps*batch_size, n_input)

    # Linear activation
    _x = tf.nn.relu(tf.matmul(_x, _weights['hidden']) + _biases['hidden'])
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _x = tf.split(_x, n_steps, 0)
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _x, dtype=tf.float32)

    # Get last time step's output feature for a "many to one" style classifier,
    # as in the image describing RNNs at the top of this page
    lstm_last_output = outputs[-1]

    # Linear activation
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']


def extract_batch_size(_train, input_step, input_batch_size):
    # Function to fetch a "batch_size" amount of data from "(x|y)_train" data.
    shape = list(_train.shape)
    shape[0] = input_batch_size
    batch_s = np.empty(shape)

    for i in range(input_batch_size):
        # Loop index
        index = ((input_step - 1) * input_batch_size + i) % len(_train)
        batch_s[i] = _train[index]

    return batch_s


def one_hot(y_):
    # Function to encode output labels from number indexes
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]

    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    # Returns floats
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]

### Build the neural network

In [ ]:
# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

# Graph weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])),  # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = lstm_rnn(x, weights, biases)

# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
)  # L2 loss prevents this overkill neural network to over fit the data
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=pred)) + l2  # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)  # Adam Optimizer

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### Train the neural network

In [ ]:
# To keep track of training's performance
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

# Launch the graph
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

# Perform Training steps with "batch_size" amount of example data at each loop
step = 1
while step * batch_size <= training_iters:
    batch_xs = extract_batch_size(x_train, step, batch_size)
    batch_ys = one_hot(extract_batch_size(y_train, step, batch_size))

    # Fit training using batch data
    _, loss, acc = sess.run(
        [optimizer, cost, accuracy],
        feed_dict={
            x: batch_xs,
            y: batch_ys
        }
    )
    train_losses.append(loss)
    train_accuracies.append(acc)

    # Evaluate network only at some steps for faster training:
    if (step * batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):
        # To not spam console, show training accuracy/loss in this "if"
        print("Training iter #" + str(step * batch_size) +
              ":   Batch Loss = " + "{:.6f}".format(loss) +
              ", Accuracy = {}".format(acc))

        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
        loss, acc = sess.run(
            [cost, accuracy],
            feed_dict={
                x: x_test,
                y: one_hot(y_test)
            }
        )
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("PERFORMANCE ON TEST SET: " +
              "Batch Loss = {}".format(loss) +
              ", Accuracy = {}".format(acc))

    step += 1

print("Optimization Finished!")


# Accuracy for test data

one_hot_predictions, accuracy, final_loss = sess.run(
    [pred, accuracy, cost],
    feed_dict={
        x: x_test,
        y: one_hot(y_test)
    }
)

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print("FINAL RESULT: " +
      "Batch Loss = {}".format(final_loss) +
      ", Accuracy = {}".format(accuracy))

### Visualization

In [ ]:
# Progress over iterations
font = {
    'family': 'DejaVu Sans',
    'weight': 'bold',
    'size': 18
}
matplotlib.rc('font', **font)

width = 12
height = 12
plt.figure(figsize=(width, height))

indep_train_axis = np.array(range(batch_size, (len(train_losses)+1)*batch_size, batch_size))
plt.plot(indep_train_axis, np.array(train_losses),     "b--", label="Train losses")
plt.plot(indep_train_axis, np.array(train_accuracies), "g--", label="Train accuracies")

indep_test_axis = np.append(
    np.array(range(batch_size, len(test_losses)*display_iter, display_iter)[:-1]),
    [training_iters]
)
plt.plot(indep_test_axis, np.array(test_losses),     "b-", label="Test losses")
plt.plot(indep_test_axis, np.array(test_accuracies), "g-", label="Test accuracies")

plt.title("Training session's progress over iterations")
plt.legend(loc='upper right', shadow=True)
plt.ylabel('Training Progress (Loss or Accuracy values)')
plt.xlabel('Training iteration')

plt.show()

![title](screenshots/har_1.png)

### Multi-class confusion matrix

In [ ]:
# Results

predictions = one_hot_predictions.argmax(1)

print("Testing Accuracy: {}%".format(100*accuracy))

print("")
print("Precision: {}%".format(100*metrics.precision_score(y_test, predictions, average="weighted")))
print("Recall: {}%".format(100*metrics.recall_score(y_test, predictions, average="weighted")))
print("f1_score: {}%".format(100*metrics.f1_score(y_test, predictions, average="weighted")))

print("")
print("Confusion Matrix:")
confusion_matrix = metrics.confusion_matrix(y_test, predictions)
print(confusion_matrix)
normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

print("")
print("Confusion matrix (normalised to % of total test data):")
print(normalised_confusion_matrix)
print("Note: training and testing data is not equally distributed amongst classes, ")
print("so it is normal that more than a 6th of the data is correctly classifier in the last category.")

# Plot Results:
width = 12
height = 12
plt.figure(figsize=(width, height))
plt.imshow(
    normalised_confusion_matrix,
    interpolation='nearest',
    cmap=plt.cm.rainbow
)
plt.title("Confusion Matrix \n(normalised to % of total test data)")
plt.colorbar()
tick_marks = np.arange(n_classes)
plt.xticks(tick_marks, LABELS, rotation=90)
plt.yticks(tick_marks, LABELS)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

![title](screenshots/har_2.png)